In [1]:
import pandas as pd
import numpy as np
import re
from bs4 import BeautifulSoup

In [2]:
question_df = pd.read_csv("Questions.csv", encoding='latin1')
tag_df = pd.read_csv("Tags.csv")
question_df.shape, tag_df.shape

((85085, 6), (244228, 2))

In [3]:
question_df.head()

,Id,OwnerUserId,CreationDate,Score,Title,Body
0,6,5.0,2010-07-19T19:14:44Z,272,The Two Cultures: statistics vs. machine learn...,"<p>Last year, I read a blog post from <a href=..."
1,21,59.0,2010-07-19T19:24:36Z,4,Forecasting demographic census,<p>What are some of the ways to forecast demog...
2,22,66.0,2010-07-19T19:25:39Z,208,Bayesian and frequentist reasoning in plain En...,<p>How would you describe in plain English the...
3,31,13.0,2010-07-19T19:28:44Z,138,What is the meaning of p values and t values i...,<p>After taking a statistics course and then t...
4,36,8.0,2010-07-19T19:31:47Z,58,Examples for teaching: Correlation does not me...,"<p>There is an old saying: ""Correlation does n..."


In [4]:
tag_df.head()

,Id,Tag
0,1,bayesian
1,1,prior
2,1,elicitation
3,2,distributions
4,2,normality


In [5]:
question_df = question_df[['Id', 'Title', 'Body']]
question_df.head()

,Id,Title,Body
0,6,The Two Cultures: statistics vs. machine learn...,"<p>Last year, I read a blog post from <a href=..."
1,21,Forecasting demographic census,<p>What are some of the ways to forecast demog...
2,22,Bayesian and frequentist reasoning in plain En...,<p>How would you describe in plain English the...
3,31,What is the meaning of p values and t values i...,<p>After taking a statistics course and then t...
4,36,Examples for teaching: Correlation does not me...,"<p>There is an old saying: ""Correlation does n..."


In [6]:
question_df['Body'] = question_df['Body'].apply(lambda x: BeautifulSoup(x).get_text())
question_df['Body'] = question_df['Body'].apply(lambda x: re.sub('[^a-zA-z]', ' ', x))
question_df['Body'] = question_df['Body'].apply(lambda x: x.lower())
question_df['Body'] = question_df['Body'].apply(lambda x: x.split())
question_df['Body'] = question_df['Body'].apply(lambda x: ' '.join(x))

In [7]:
question_df.sample(6)

,Id,Title,Body
37370,225960,Analysis of small set of interview data using ...,i have companies response about questions para...
32321,114141,why is the denominator of the correlation coef...,i don t quite understand what is going on in t...
20616,213843,Fitting a cross-classified variance components...,i am attempting to fit a multivariate variance...
53616,51988,How to estimate Cohen's D using the independen...,i want to calculate cohen s d with confidence ...
20872,5062,Geometric circle fitting with known radius,i have data points from a half circle and i al...
18814,181130,Ensemble LDA on different feature spaces?,i m working on a classification problem where ...


In [8]:
tag_df = tag_df.groupby('Id').apply(lambda x: x['Tag'].values).reset_index(name='tags')
tag_df.head()

/tmp/ipython-input-2564490889.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  tag_df = tag_df.groupby('Id').apply(lambda x: x['Tag'].values).reset_index(name='tags')


,Id,tags
0,1,"[bayesian, prior, elicitation]"
1,2,"[distributions, normality]"
2,3,"[software, open-source]"
3,4,"[distributions, statistical-significance]"
4,6,[machine-learning]


In [9]:
df = pd.merge(question_df, tag_df)

In [10]:
df.head()

,Id,Title,Body,tags
0,6,The Two Cultures: statistics vs. machine learn...,last year i read a blog post from brendan o co...,[machine-learning]
1,21,Forecasting demographic census,what are some of the ways to forecast demograp...,"[forecasting, population, census]"
2,22,Bayesian and frequentist reasoning in plain En...,how would you describe in plain english the ch...,"[bayesian, frequentist]"
3,31,What is the meaning of p values and t values i...,after taking a statistics course and then tryi...,"[hypothesis-testing, t-test, p-value, interpre..."
4,36,Examples for teaching: Correlation does not me...,there is an old saying correlation does not me...,"[correlation, teaching]"


In [11]:
freq = {}
for i in df['tags']:
  for j in i:
    if j in freq.keys():
      freq[j] += 1
    else:
      freq[j] = 1

freq = dict(sorted(freq.items(), key=lambda x: x[1] , reverse=True))

In [12]:
common_tags = list(freq.keys())[:20]
common_tags

['r',
 'regression',
 'machine-learning',
 'time-series',
 'probability',
 'hypothesis-testing',
 'self-study',
 'distributions',
 'logistic',
 'classification',
 'correlation',
 'statistical-significance',
 'bayesian',
 'anova',
 'normal-distribution',
 'multiple-regression',
 'mixed-model',
 'clustering',
 'neural-networks',
 'mathematical-statistics']

In [13]:
x = []
y = []

for i in range(len(df['tags'])):
  temp = []
  for j in df['tags'][i]:
    if j in common_tags:
      temp.append(j)

  if len(temp) > 1:
    x.append(df['Body'][i])
    y.append(temp)

In [14]:
len(x)

19689

In [15]:
y[:20]

[['r', 'time-series'],
 ['regression', 'distributions'],
 ['r', 'clustering'],
 ['time-series', 'neural-networks'],
 ['probability', 'bayesian'],
 ['distributions', 'normal-distribution'],
 ['distributions',
  'probability',
  'hypothesis-testing',
  'mathematical-statistics'],
 ['self-study', 'mathematical-statistics'],
 ['distributions', 'bayesian'],
 ['correlation', 'self-study'],
 ['regression', 'correlation', 'mathematical-statistics'],
 ['hypothesis-testing', 'self-study'],
 ['r', 'regression', 'time-series'],
 ['hypothesis-testing', 'mathematical-statistics'],
 ['r', 'time-series', 'self-study'],
 ['probability', 'hypothesis-testing'],
 ['r', 'regression'],
 ['r', 'mixed-model'],
 ['r', 'regression', 'anova'],
 ['regression', 'logistic', 'multiple-regression']]

In [16]:
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer()
y = mlb.fit_transform(y)

print(y)

[[0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [1 0 0 ... 1 0 0]
 [0 0 0 ... 0 1 0]
 [0 0 0 ... 1 0 0]]


In [17]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42, shuffle=True)

In [18]:
y_test[0]

array([0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [19]:
from tensorflow.keras.preprocessing.text import Tokenizer

X_tokenizer = Tokenizer()
X_tokenizer.fit_on_texts(X_train)
len(X_tokenizer.word_index)

35803

In [20]:
tresh = 3
cnt = 0

for key, val in X_tokenizer.word_counts.items():
  if val >= tresh:
    cnt += 1
cnt

17443

In [21]:
X_tokenizer = Tokenizer(num_words=cnt, oov_token='unk')
X_tokenizer.fit_on_texts(X_train)
len(X_tokenizer.word_index)

35804

In [22]:
max_len = 100

X_train_seq = X_tokenizer.texts_to_sequences(X_train)
X_test_seq = X_tokenizer.texts_to_sequences(X_test)

In [23]:
from keras.preprocessing.sequence import pad_sequences

X_train_seq = pad_sequences(X_train_seq, padding='post', maxlen=max_len)
X_test_seq = pad_sequences(X_test_seq, padding='post', maxlen=max_len)
X_test_seq

array([[ 165,   18,   82, ...,   32, 2253,   81],
       [  22,  187,    5, ...,   63,    3,  205],
       [ 129,   64,  149, ...,    0,    0,    0],
       ...,
       [  44, 5852,   11, ...,   57,   52,   19],
       [6652, 2518, 2248, ..., 2518, 4761, 6652],
       [   4,   15,   78, ...,   14,   22, 3558]], dtype=int32)

In [24]:
from keras.layers import *
from keras.models import *
from keras.callbacks import *

In [25]:
x_voc_size = X_tokenizer.num_words + 1

In [26]:
model = Sequential()
model.add(Embedding(input_dim=x_voc_size, output_dim=50, trainable=True, input_shape=(max_len, ), mask_zero=True))
model.add(Conv1D(64, 3, padding='same'))
model.add(Dropout(rate=0.01))
model.add(GlobalMaxPool1D())
model.add(Dense(units=128, activation='relu'))
model.add(Dense(units=20, activation='sigmoid'))

model.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:100: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/usr/local/lib/python3.12/dist-packages/keras/src/layers/layer.py:965: UserWarning: Layer 'conv1d' (of type Conv1D) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 100, 50)        │       872,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d (Conv1D)                 │ (None, 100, 64)        │         9,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 100, 64)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling1d            │ (None, 64)             │             0 │
│ (GlobalMaxPooling1D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │         8,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 20)             │         2,580 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 892,764 (3.41 MB)

 Trainable params: 892,764 (3.41 MB)

 Non-trainable params: 0 (0.00 B)

In [27]:
from tensorflow.keras import metrics

model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=[
        metrics.Precision(name='precision'),
        metrics.Recall(name='recall')
    ]
)


In [28]:
checkpoint = ModelCheckpoint('best_weighs.keras', monitor='val_loss', verbose=1, save_best_only=True, mode='min')

In [29]:
model.fit(X_train_seq, y_train, validation_data=(X_test_seq, y_test), batch_size=64, epochs=20, callbacks=[checkpoint])

Epoch 1/20
246/247 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 0.4202 - precision: 0.1653 - recall: 0.0673
Epoch 1: val_loss improved from inf to 0.27482, saving model to best_weighs.keras
247/247 ━━━━━━━━━━━━━━━━━━━━ 10s 29ms/step - loss: 0.4195 - precision: 0.1668 - recall: 0.0672 - val_loss: 0.2748 - val_precision: 0.7750 - val_recall: 0.2046
Epoch 2/20
245/247 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 0.2560 - precision: 0.7492 - recall: 0.2640
Epoch 2: val_loss improved from 0.27482 to 0.22477, saving model to best_weighs.keras
247/247 ━━━━━━━━━━━━━━━━━━━━ 8s 32ms/step - loss: 0.2558 - precision: 0.7492 - recall: 0.2645 - val_loss: 0.2248 - val_precision: 0.7697 - val_recall: 0.3556
Epoch 3/20
247/247 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 0.2109 - precision: 0.7661 - recall: 0.4052
Epoch 3: val_loss improved from 0.22477 to 0.20727, saving model to best_weighs.keras
247/247 ━━━━━━━━━━━━━━━━━━━━ 7s 28ms/step - loss: 0.2109 - precision: 0.7661 - recall: 0.4053 - val_loss: 0.2073 -

In [30]:
model.load_weights('best_weighs.keras')

In [31]:
pred_prob = model.predict(X_test_seq)
np.round(pred_prob[0], 2)

124/124 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step


array([0.01, 0.  , 0.55, 0.09, 0.  , 0.  , 0.  , 0.03, 0.64, 0.  , 0.  ,
       0.  , 0.09, 0.  , 0.01, 0.32, 0.02, 0.02, 0.02, 0.02],
      dtype=float32)

In [32]:
threshold = np.arange(0, 0.5, 0.01)
threshold

array([0.  , 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1 ,
       0.11, 0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2 , 0.21,
       0.22, 0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3 , 0.31, 0.32,
       0.33, 0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.4 , 0.41, 0.42, 0.43,
       0.44, 0.45, 0.46, 0.47, 0.48, 0.49])

In [33]:
def classify(pred_pro, tsh):
  y_pred_seq = []

  for i in pred_pro:
    temp = []
    for j in i:
      if j>=tsh:
        temp.append(1)
      else:
        temp.append(0)
    y_pred_seq.append(temp)

  return y_pred_seq

In [34]:
from sklearn.metrics import *

score = []
y_true = np.array(y_test)

for thresh in threshold:
  y_pred_seq = classify(pred_prob, thresh)
  y_pred = np.array(y_pred_seq)
  score.append(f1_score(y_true, y_pred, average='micro'))

In [35]:
opt = threshold[score.index(max(score))]
opt

np.float64(0.36)

In [36]:
y_true = np.array(y_test)
y_pred = classify(pred_prob, opt)
y_pred = np.array(y_pred)

y_pred[0], y_true[0]

(array([0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 array([0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]))

In [37]:
y_pred[1], y_true[1]

(array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]),
 array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]))

In [38]:
y_pred[23], y_true[23]

(array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]),
 array([0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]))

In [39]:
y_pred[678], y_true[678]

(array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1]),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0]))

In [40]:
precision = precision_score(y_true, y_pred, average='micro', zero_division=0)
recall = recall_score(y_true, y_pred, average='micro', zero_division=0)
f1 = f1_score(y_true, y_pred, average='micro', zero_division=0)

print("Overall Performance Summary:")
print(f"Precision: {precision:.4f}")
print(f"Recall:    {recall:.4f}")
print(f"F1-score:  {f1:.4f}")

Overall Performance Summary:
Precision: 0.6740
Recall:    0.6159
F1-score:  0.6436


In [41]:
#SIMPLE RNN

# Overall Performance Summary:
# Precision: 0.3965
# Recall:    0.5033
# F1-score:  0.4436

In [42]:
#LSTM

# Overall Performance Summary:
# Precision: 0.5726
# Recall:    0.5967
# F1-score:  0.5844

# (array([0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
#  array([0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]))

In [43]:
#GRU

# Overall Performance Summary:
# Precision: 0.6103
# Recall:    0.6037
# F1-score:  0.6070

# (array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]),
#  array([0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]))

In [ ]:
#CNN

# Overall Performance Summary:
# Precision: 0.6740
# Recall:    0.6159
# F1-score:  0.6436

# (array([0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
#  array([0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]))